# Testing out GCP billing export

In [3]:
from google.cloud import bigquery
import os
import tqdm
import numpy as np
import pandas as pd
import pandas_gbq
from config import My_Config as cfg

In [6]:
credential_path = cfg.gcp_credential_path()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
project_id='gcp-prices-358310'
dataset_location='europe-west4'

In [3]:
client = bigquery.Client(project=project_id, location=dataset_location)

In [7]:
query = """SELECT * FROM `gcp-prices-358310.12345.gcp_billing_export_v1_017DA2_854255_CCEB23`"""
data = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████| 626/626 [00:00<00:00, 1050.98rows/s]


In [8]:
df = pd.DataFrame(data)
len_row, len_col = df.shape

In [10]:
#Make a directory if it doesn't exist
if not os.path.exists('gcp'):
    os.makedirs('gcp')

df.to_csv('gcp/GCP-Billing-Data.csv', index=False)
df.to_parquet('gcp/GCP-Billing-Data.parquet', index=False)